In [ ]:
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env
load_dotenv()

class GithubApis:
    def __init__(self):
        # GitHub username and repository
        self.username = "neonwatty"
        self.repository = "meme-search"

        # Load the personal access token from .env
        self.access_token = os.getenv("GITHUB_TOKEN")
        if not self.access_token:
            raise ValueError("GitHub token not found. Please set GITHUB_TOKEN in the .env file.")

        # Form headers with authentication
        self.headers = {
            "Authorization": f"token {self.access_token}",
            "Accept": "application/vnd.github+json"
        }

    def form_url(self, endpoint):
        # views fetched in backward 14-day window from fetch date
        # GitHub API URL for traffic views
        url = f"https://api.github.com/repos/{self.username}/{self.repository}/{endpoint}"
        return url
        
    def get_traffic(self):
        # form url
        endpoint = '/traffic/views'
        url = self.form_url(endpoint)

        # Make the GET request
        response = requests.get(url, headers=self.headers)

        # Check the response
        if response.status_code == 200:
            self.traffic_data = response.json()
            return self.traffic_data
        else:
            print(f"Failed to fetch traffic data: {response.status_code}")
            print(response.json())

In [3]:
data = fetch_traffic_views()

In [8]:
data

{'count': 182,
 'uniques': 55,
 'views': [{'timestamp': '2024-12-28T00:00:00Z', 'count': 33, 'uniques': 5},
  {'timestamp': '2024-12-29T00:00:00Z', 'count': 4, 'uniques': 2},
  {'timestamp': '2024-12-30T00:00:00Z', 'count': 3, 'uniques': 3},
  {'timestamp': '2024-12-31T00:00:00Z', 'count': 2, 'uniques': 2},
  {'timestamp': '2025-01-01T00:00:00Z', 'count': 8, 'uniques': 5},
  {'timestamp': '2025-01-02T00:00:00Z', 'count': 14, 'uniques': 10},
  {'timestamp': '2025-01-03T00:00:00Z', 'count': 5, 'uniques': 5},
  {'timestamp': '2025-01-04T00:00:00Z', 'count': 10, 'uniques': 4},
  {'timestamp': '2025-01-05T00:00:00Z', 'count': 12, 'uniques': 8},
  {'timestamp': '2025-01-06T00:00:00Z', 'count': 5, 'uniques': 3},
  {'timestamp': '2025-01-07T00:00:00Z', 'count': 38, 'uniques': 2},
  {'timestamp': '2025-01-08T00:00:00Z', 'count': 11, 'uniques': 6},
  {'timestamp': '2025-01-09T00:00:00Z', 'count': 6, 'uniques': 3},
  {'timestamp': '2025-01-10T00:00:00Z', 'count': 5, 'uniques': 4},
  {'timestamp':

In [7]:
sum([v['uniques'] for v in data['views']])

64